In [1]:
from PyQt5.QtWidgets import QFileDialog, QApplication
import os
import os.path as ospath
import numpy as np
import pandas as pd

def gui_fname(dir=None):
    """
    Select a file via a dialog and return the file name.
    """
    if dir is None: 
        dir ="./"

    app = QApplication([dir])
    fname = QFileDialog.getExistingDirectory(None, "Select a folder...", 
            dir)
    if isinstance(fname, tuple):
        return fname[0]
    else: 
        return str(fname)
    
def load_ring_data_df(dirname, filename):
    file = ospath.join(dirname, filename)

    try: 
        df = pd.read_pickle(file)
    except FileNotFoundError:
        print("No results of previously analyzed datasets were detected.")
        return None
    else: 
        print("Results of previously analyzed datasets were detected.")
        return df

def load_files(dirname):
    
    os.chdir(dirname)
    files = glob.glob("*.hdf5")
    
    if files:
        print("{} HDF5 files found.".format(len(files)))
    else:
        print("No HDF5 files found at: {}".format(dirname))
            
    return files
    
def load_data(path):

    try:
        locs, info = io.load_locs(path)
    except io.NoMetadataFileError:
        return None, None, None
    
    try:
        pixelsize = info[1]["Pixelsize"]
    except:  
        print("No pixelsize found in yaml file. Default 130 nm used.")

        
    return locs, info, pixelsize

    
def save_ring_locs(locs, info, path, file_id, pick, id, link=False):
    

    ending = ".hdf5"
       
    locs.x /= 130
    locs.y /= 130
    
    locs_name = "file_{}_pick_{}_ring_{}{}".format(file_id, pick, id, ending)
    locs_path = os.path.join(path,"ring_locs")
    locs_path_name = os.path.join(locs_path, locs_name)
    
    if not os.path.isdir(locs_path):
        os.makedirs(locs_path)
    
    io.save_locs(locs_path_name, locs, info)

In [19]:
#path = gui_fname()
path = r'W:\users\reinhardt\z.software\Git\spor-PAINT\dev_sr\spor-paint\FtsZ\FtsZ_picks_all'

#filenames = load_files(path)

df_ring_data = load_ring_data_df(path, "ring_data.pkl")

print(df_ring_data.keys())

radius_step = 15 # nm
sub_step = 5


radius_step = 30 # nm
sub_step = 30

Results of previously analyzed datasets were detected.
Index(['fov_id', 'cell_type', 'filename', 'group', 'radius',
       'angle_between_ring_and_coverslip', 'n_events', 'mean_bright',
       'mean_dark', 'fit_bright', 'fit_dark'],
      dtype='object')


In [13]:
"""
radius groups: resolution
rolling average

NeNAs:
5.9
5.82
4.37
3.97
4.16
9.1
"""

'\nradius groups: resolution\nrolling average\n\nNeNAs:\n5.9\n5.82\n4.37\n3.97\n4.16\n9.1\n'

In [20]:
min_radius_bin = df_ring_data['radius'].min()
print(min_radius_bin)

min_radius_bin = int(min_radius_bin / sub_step) * sub_step
print(min_radius_bin)

max_radius_bin = df_ring_data['radius'].max()
print(max_radius_bin)
max_radius_bin = (int(max_radius_bin / sub_step)+1) * sub_step
print(max_radius_bin)

bins = []
bins.append([min_radius_bin, min_radius_bin+radius_step])
i = 0
bin_end = bins[0][1]
while bin_end < max_radius_bin:
    
    bin_start = bins[i][0]+sub_step
    bin_end = bins[i][1]+sub_step
    bins.append([bin_start, bin_end])
    i+=1
    
print(bins)


95.879150390625
90
1617.4049072265625
1620
[[90, 120], [120, 150], [150, 180], [180, 210], [210, 240], [240, 270], [270, 300], [300, 330], [330, 360], [360, 390], [390, 420], [420, 450], [450, 480], [480, 510], [510, 540], [540, 570], [570, 600], [600, 630], [630, 660], [660, 690], [690, 720], [720, 750], [750, 780], [780, 810], [810, 840], [840, 870], [870, 900], [900, 930], [930, 960], [960, 990], [990, 1020], [1020, 1050], [1050, 1080], [1080, 1110], [1110, 1140], [1140, 1170], [1170, 1200], [1200, 1230], [1230, 1260], [1260, 1290], [1290, 1320], [1320, 1350], [1350, 1380], [1380, 1410], [1410, 1440], [1440, 1470], [1470, 1500], [1500, 1530], [1530, 1560], [1560, 1590], [1590, 1620]]


In [26]:
"""
grid pattern

load rotated locs

move to grid positions

test version with horizontal ring



"""

for bin in bins:
    bin_start = bin[0]
    bin_end = bin[1]
    
    rings = df_ring_data[(df_ring_data['radius']>=bin_start) & (df_ring_data['radius']<bin_end)]
    print(bin)
    print(rings['radius'])
    
    if len(rings) < 5:
        continue
    
    

[90, 120]
1      109.834351
62     107.389099
140    116.719170
154     97.937881
172    116.695793
201    111.563461
245    115.507103
288    112.976562
305    115.529556
349    107.284332
361     95.879150
405    115.215111
414    111.463249
Name: radius, dtype: float32
[120, 150]
4      144.474365
19     139.194275
73     131.302399
88     123.725204
101    135.056671
110    142.570648
119    130.619751
130    140.875153
132    144.927399
135    130.424332
138    137.370056
145    140.638611
162    136.133514
169    124.187469
170    145.474457
180    123.408409
182    124.519226
189    145.705978
215    149.017914
223    123.261795
251    135.484497
263    141.880341
276    133.918549
286    126.788101
290    122.294861
318    148.149933
339    145.226349
340    147.842499
355    135.586975
368    146.007324
375    134.785004
389    134.881897
432    136.426056
434    137.303406
Name: radius, dtype: float32
[150, 180]
2      151.573853
3      165.349213
8      172.832397
17     151